## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(12)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ponta Do Sol,PT,32.67,-17.10,75.00,73,20,12.75,few clouds
1,1,Arraial Do Cabo,BR,-22.97,-42.03,69.80,94,90,12.75,mist
2,2,Avarua,CK,-21.21,-159.78,73.40,78,100,14.99,overcast clouds
3,3,Purepero,MX,19.92,-102.02,60.80,87,75,3.13,heavy intensity rain
4,4,Busselton,AU,-33.65,115.33,64.00,65,100,11.36,light rain
5,5,Qaanaaq,GL,77.48,-69.36,12.97,87,99,7.18,overcast clouds
6,6,Beloha,MG,-25.17,45.05,76.93,53,0,14.03,clear sky
7,7,Port Macquarie,AU,-31.43,152.92,61.00,81,3,6.93,clear sky
8,8,Xucheng,CN,20.35,110.13,73.40,83,0,4.47,light rain
9,9,Rikitea,PF,-23.12,-134.97,74.82,71,12,7.49,few clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 85


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_new_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp)]
cities_new_df.head(12)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ponta Do Sol,PT,32.67,-17.10,75.00,73,20,12.75,few clouds
1,1,Arraial Do Cabo,BR,-22.97,-42.03,69.80,94,90,12.75,mist
2,2,Avarua,CK,-21.21,-159.78,73.40,78,100,14.99,overcast clouds
3,3,Purepero,MX,19.92,-102.02,60.80,87,75,3.13,heavy intensity rain
4,4,Busselton,AU,-33.65,115.33,64.00,65,100,11.36,light rain
6,6,Beloha,MG,-25.17,45.05,76.93,53,0,14.03,clear sky
7,7,Port Macquarie,AU,-31.43,152.92,61.00,81,3,6.93,clear sky
8,8,Xucheng,CN,20.35,110.13,73.40,83,0,4.47,light rain
9,9,Rikitea,PF,-23.12,-134.97,74.82,71,12,7.49,few clouds
10,10,Goianapolis,BR,-16.51,-49.02,77.00,78,90,6.93,light rain


In [21]:
# 4a. Determine if there are any empty rows.
cities_new_df.isnull().count()
## No empty rows

City_ID        365
City           365
Country        365
Lat            365
Lng            365
Max Temp       365
Humidity       365
Cloudiness     365
Wind Speed     365
Description    365
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= cities_new_df.dropna()

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(12)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,75.00,few clouds,32.67,-17.10,
1,Arraial Do Cabo,BR,69.80,mist,-22.97,-42.03,
2,Avarua,CK,73.40,overcast clouds,-21.21,-159.78,
3,Purepero,MX,60.80,heavy intensity rain,19.92,-102.02,
4,Busselton,AU,64.00,light rain,-33.65,115.33,
6,Beloha,MG,76.93,clear sky,-25.17,45.05,
7,Port Macquarie,AU,61.00,clear sky,-31.43,152.92,
8,Xucheng,CN,73.40,light rain,20.35,110.13,
9,Rikitea,PF,74.82,few clouds,-23.12,-134.97,
10,Goianapolis,BR,77.00,light rain,-16.51,-49.02,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lats = row["Lat"]
    lngs = row["Lng"]
    params["location"] = f"{lats},{lngs}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:

# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))